In [ ]:
import librosa
import numpy as np
import scipy.stats as stats
import IPython.display
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# オリジナル音源データの読み込み/Load original sound data

!git clone https://github.com/shinolab/fukuzatu_jikken.git -b data

fs = 44100
orig_wav_1, _ = librosa.load('./fukuzatu_jikken/common_voice_ja_39589161.mp3', sr=fs, offset=1.0, duration=3.0)
orig_wav_2, _ = librosa.load('./fukuzatu_jikken/common_voice_ja_39589175.mp3', sr=fs, offset=1.0, duration=3.0)

In [ ]:
IPython.display.Audio(data=orig_wav_1, rate=fs)

In [ ]:
IPython.display.Audio(data=orig_wav_2, rate=fs)

# 演習1: 音声データを可視化してみよう
# Excersise 1: Let's visualize the audio data

In [ ]:
# hitn: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

# ここにプロットするコードを書いてください/Write code to plot here

# 演習2: 音声データの強度分布を見てみよう
# Excersise 2: Let's see the intensity distribution of the audio data

In [ ]:
# hitn: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html

# ここにヒストグラムをプロットするコードを書いてください/Write code to plot a histogram here
bins = 100

# 以下はガウス分布をプロットするコードです/The following code plots a Gaussian distribution
mean = np.mean(orig_wav_1)
std = np.std(orig_wav_1)
gaussian = stats.norm.pdf(np.linspace(-1, 1, 100), mean, std)
plt.plot(np.linspace(-1, 1, 100), gaussian);

# 演習3: 混合前/混合後の信号の散布図を見てみよう
# Excersise 3: Let's see the scatter plot of the signals before and after mixing

In [ ]:
# 音源データのミキシング/Mixing sound data
orig_wav = np.stack([orig_wav_1, orig_wav_2], axis = 1).T
A = np.array([[0.92, 0.31], [0.41, 0.73]])

mix_wav = np.dot(A, orig_wav)
mix_wav_1, mix_wav_2 = mix_wav[0], mix_wav[1]

In [ ]:
IPython.display.Audio(data=mix_wav_1, rate=fs)

In [ ]:
IPython.display.Audio(data=mix_wav_2, rate=fs)

In [ ]:
# hint: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html

# ここに散布図をプロットするコードを書いてください/Write code to plot a scatter plot here

plt.xlim(-1, 1)
plt.ylim(-1, 1);

# 演習4: 白色化してみよう
# Excersise 4: Let's whiten the signals

In [ ]:
# 共分散行列の計算/Calculation of covariance matrix
def covariace_matrix(X):
    X_mu = np.mean(X, axis=1).reshape(2, 1)
    X_cov = np.dot((X - X_mu), (X - X_mu).T) / X.shape[1]
    return X_cov

# 白色化/whitening
# 現在は、白色化の関数が未実装です。以下の関数を完成させてください。
# Currently, the whitening function is not implemented. Please complete the following function.
def whitening(X):
    return X

X_dash = whitening(mix_wav)
print("Mixed:", covariace_matrix(mix_wav))
print("Whitened:", covariace_matrix(X_dash))

plt.scatter(X_dash[0] / np.amax(X_dash[0]), X_dash[1] / np.amax(X_dash[1]))

plt.xlim(-1, 1)
plt.ylim(-1, 1);

# 演習5: 尖度を計算してみよう
# Excersise 5: Let's calculate the kurtosis

In [ ]:
# 尖度の計算/Calculate kurtosis
# 現在は、尖度の計算の関数が未実装です。以下の関数を完成させてください。
# Currently, the function for calculating kurtosis is not implemented. Please complete the following function.
def kurtosis(x):
    return 0

# 各角度に対して尖度を計算/Calculate kurtosis for each angle
# ナイーブに全探索していますが, 解析解もあります, 興味があれば考えてみてください
# We are naively searching for all angles, but there is an analytical solution. If you are interested, please think about it.
step = 1
degrees = np.arange(0, 89, step)
kurtoses = np.zeros(len(degrees))
for i, deg in enumerate(degrees):
    theta = deg / 180 * np.pi
    R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta),  np.cos(theta)]])
    x = np.dot(R, X_dash)
    kurtoses[i] = kurtosis(x[0]) + kurtosis(x[1])

plt.plot(degrees, kurtoses)

# 最大尖度を持つ角度/The angle with the maximum kurtosis
deg = np.argmax(kurtoses) * step

In [ ]:
# 推定した回転角で分離/Separate signals from the estimated rotation angle 
theta = deg / 180 * np.pi
R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta),  np.cos(theta)]])
estimated = np.dot(R, X_dash)

plt.scatter(estimated[0], estimated[1])

In [ ]:
IPython.display.Audio(data=estimated[0], rate=fs)

In [ ]:
IPython.display.Audio(data=estimated[1], rate=fs)